In [1]:
from terrainbento import Basic

params = {
    # create the Clock.
    "clock": {"start": 0, 
              "step": 10,
              "stop": 1e5},
    
    # Create the Grid.
    "grid": {"grid": {"RasterModelGrid":[(200, 320), {"xy_spacing": 10}]},
             "fields": {"at_node": {"topographic__elevation":{"random":[{"where":"CORE_NODE"}]}}}},
    
    # Set up Boundary Handlers
    "boundary_handlers":{"NotCoreNodeBaselevelHandler": {"modify_core_nodes": True, 
                                                         "lowering_rate": -0.001}},
    # Parameters that control output.
    "output_interval": 1e3, 
    "save_first_timestep": True,
    "fields":["topographic__elevation"],
             
    # Parameters that control process and rates.
    "water_erodibility" : 0.001,
    "m_sp" : 0.5,
    "n_sp" : 1.0, 
    "regolith_transport_parameter" : 0.2,           
         }

model = Basic.from_dict(params)

In [2]:
model.run()

In [3]:
from landlab import imshow_grid

filenames = []
ds = model.to_xarray_dataset()
for i in range(ds.topographic__elevation.shape[0]):
    filename = "temp_output."+str(i)+".png"
    imshow_grid(model.grid, 
                ds.topographic__elevation.values[i, :, :], 
                cmap="viridis", 
                limits=(0, 12), 
                output=filename)
    filenames.append(filename)

<Figure size 432x288 with 0 Axes>

In [4]:
import os
import imageio
with imageio.get_writer("terrainbento_example.gif", mode="I") as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        os.remove(filename)

In [5]:
model.remove_output_netcdfs()